In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from stop_words import get_stop_words
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import gensim
from gensim.models import word2vec
from sklearn.model_selection import cross_val_predict
import re
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')

from keras.layers import Dense, Input, Embedding, Dropout, GRU, Bidirectional
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
ads = pd.read_csv('/Users/admin/Documents/contextAdvertising1.csv')

In [3]:
ads.shape

(799999, 3)

In [4]:
ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799999 entries, 0 to 799998
Data columns (total 3 columns):
atitle     799999 non-null object
atext      799983 non-null object
adomain    799999 non-null object
dtypes: object(3)
memory usage: 18.3+ MB


In [5]:
ads.head()

,atitle,atext,adomain
0,Клуб активного отдыха «0.67»,Детский пейнтбол. Спортивный пейнтбол. Тактиче...,0-67.relax.by
1,Антигравитационный чехол IPhone 5,Успейте купить антигравитационный чехол для IP...,0-antigravity.ru
2,Антигравитационный чехол купить!,Антигравитационный чехол для телефона купить з...,0-antigravity.ru
3,Беспроцентный заем от Moneyveo,Без справок! Получите до 3 000 грн. на карту п...,0-credit.moneyveo.ua
4,Беспроцентный заем сотруднику,Акция! Получите Кредит Онлайн под 0%. Без Спра...,0-credit.moneyveo.ua


In [6]:
#include all the punctuation signs\n",
def punctuation_remover(row):
    punctuation = '!"#$&\'()*+,-./:;<=>?@[\\]^_`{|}~«»'
    
    punctuationNoPeriod = "[" + re.sub("\.","", punctuation) + "]"
    row = re.sub(punctuationNoPeriod, " ", str(row))
    #to remove double white spaces and create space after %,
    row = row.replace('  ', ' ')
    row = row.replace('%', '% ')
    return row.strip()

In [7]:
# remove punctuation from the ads text
ads['atext'] = ads['atext'].apply(punctuation_remover)

In [8]:
# remove punctuation from the ads text
ads['atitle'] = ads['atext'].apply(punctuation_remover)

In [9]:
ads.nunique()

atitle     348184
atext      348409
adomain     19181
dtype: int64

In [10]:
(ads['atitle'] + ads['atext']).nunique()

348409

In [11]:
unique =ads.drop_duplicates(subset = ['atext'], keep=False)

In [12]:
unique =unique.drop_duplicates(subset = ['atitle'], keep=False)

In [13]:
unique= unique.drop_duplicates(keep=False)

In [14]:
unique.nunique()

atitle     320632
atext      320632
adomain      4818
dtype: int64

In [15]:
unique.shape

(320632, 3)

In [16]:
unique.head()

,atitle,atext,adomain
0,Детский пейнтбол Спортивный пейнтбол Тактическ...,Детский пейнтбол Спортивный пейнтбол Тактическ...,0-67.relax.by
1,Успейте купить антигравитационный чехол для IP...,Успейте купить антигравитационный чехол для IP...,0-antigravity.ru
2,Антигравитационный чехол для телефона купить з...,Антигравитационный чехол для телефона купить з...,0-antigravity.ru
3,Без справок Получите до 3 000 грн на карту под...,Без справок Получите до 3 000 грн на карту под...,0-credit.moneyveo.ua
4,Акция Получите Кредит Онлайн под 0% Без Справ...,Акция Получите Кредит Онлайн под 0% Без Справ...,0-credit.moneyveo.ua


In [17]:
unique.index

Int64Index([     0,      1,      2,      3,      4,     10,     11,     12,
                13,     14,
            ...
            772509, 772510, 772512, 772515, 772516, 772517, 772518, 772519,
            772520, 772521],
           dtype='int64', length=320632)

In [18]:
unique = unique.reset_index(drop=True)

In [19]:
def text_parser(series): 
    series_copy = series.copy()
    list_of_strings = []
    for row in series:
        row= row.lower().split()      
        list_of_strings.append(row)
        series_copy=list_of_strings
    return series_copy

In [20]:
for entry in unique['atitle'].iloc[:3]:
    print(entry)

Детский пейнтбол Спортивный пейнтбол Тактический пейнтбол Выездные игры
Успейте купить антигравитационный чехол для IPhone Второй всего за 990 руб
Антигравитационный чехол для телефона купить здесь Успейте купить за 1990р


In [21]:
split_ads = unique.copy()

In [22]:
split_ads.shape

(320632, 3)

In [23]:
split_ads['atitle'] = text_parser(unique['atitle'])

In [24]:
split_ads['atext'] = text_parser(unique['atext'])

In [25]:
split_ads.head()

,atitle,atext,adomain
0,"[детский, пейнтбол, спортивный, пейнтбол, такт...","[детский, пейнтбол, спортивный, пейнтбол, такт...",0-67.relax.by
1,"[успейте, купить, антигравитационный, чехол, д...","[успейте, купить, антигравитационный, чехол, д...",0-antigravity.ru
2,"[антигравитационный, чехол, для, телефона, куп...","[антигравитационный, чехол, для, телефона, куп...",0-antigravity.ru
3,"[без, справок, получите, до, 3, 000, грн, на, ...","[без, справок, получите, до, 3, 000, грн, на, ...",0-credit.moneyveo.ua
4,"[акция, получите, кредит, онлайн, под, 0%, без...","[акция, получите, кредит, онлайн, под, 0%, без...",0-credit.moneyveo.ua


In [26]:
split_ads['combined'] = split_ads['atext'] +split_ads['atitle']

In [27]:
model_w2v = word2vec.Word2Vec(
    split_ads['combined'],
    workers=-1,     
    min_count=5,   
    window=30,      
    sg=0,          
    sample=1e-3 ,  
    size=300,      
    hs=1           
)

In [28]:
vocabulary = model_w2v.wv.vocab.keys()
print(len(vocabulary))

44011


In [29]:
word_vectors = model_w2v.wv

In [30]:
X = split_ads['atext']

In [31]:
X_train, X_test = train_test_split(X, test_size=0.3, random_state=0)

In [32]:
len(X_train)

224442

In [33]:
len(X_test)

96190

In [34]:
MAX_NB_WORDS = len(vocabulary)

In [35]:
MAX_NB_WORDS

44011

In [36]:
from collections import Counter
vocab=Counter()

for entry in X_train:
    vocab.update(entry)

In [37]:
word_index = {t[0]: i+1 for i,t in enumerate(vocab.most_common(MAX_NB_WORDS))}

In [38]:
training_sequences = [[word_index.get(t, 0) for t in ad_text]
             for ad_text in X_train]

In [39]:
test_sequences = [[word_index.get(t, 0) for t in ad_text]
             for ad_text in X_test]

In [40]:
max([len(x) for x in training_sequences])

22

In [41]:
max([len(x) for x in test_sequences])

19

In [42]:
MAX_SEQUENCE_LENGTH = max([len(x) for x in training_sequences])

In [43]:
def ngram_sequence_generator(input_text):
    ngram_inputs = []
    for entry in input_text:
        for i in range(1, len(entry)):
            ngram_seq = entry[:i+1]
            ngram_inputs.append(ngram_seq)
            
    return ngram_inputs

In [44]:
ngram_input_sequences_training = ngram_sequence_generator(training_sequences)

In [45]:
ngram_input_sequences_test = ngram_sequence_generator(test_sequences)

In [46]:
ngram_input_sequences_training[:20]

[[183, 44],
 [183, 44, 4],
 [183, 44, 4, 20],
 [183, 44, 4, 20, 1643],
 [183, 44, 4, 20, 1643, 15680],
 [183, 44, 4, 20, 1643, 15680, 6],
 [183, 44, 4, 20, 1643, 15680, 6, 21],
 [183, 44, 4, 20, 1643, 15680, 6, 21, 941],
 [171, 1],
 [171, 1, 30],
 [171, 1, 30, 45],
 [171, 1, 30, 45, 346],
 [171, 1, 30, 45, 346, 354],
 [171, 1, 30, 45, 346, 354, 5],
 [171, 1, 30, 45, 346, 354, 5, 1038],
 [171, 1, 30, 45, 346, 354, 5, 1038, 3589],
 [298, 2440],
 [298, 2440, 574],
 [298, 2440, 574, 14],
 [298, 2440, 574, 14, 4]]

In [47]:
from keras.utils import to_categorical
def generate_padded_sequence(input_sequences):
    input_sequences = pad_sequences(input_sequences, maxlen=MAX_SEQUENCE_LENGTH, 
                     padding="pre", truncating="post")
    

    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = to_categorical(label, num_classes=MAX_NB_WORDS+1)
    return predictors, label

In [48]:
predictors_tr, label_tr = generate_padded_sequence(ngram_input_sequences_training)

In [49]:
predictors_test, label_test = generate_padded_sequence(ngram_input_sequences_test)

In [50]:
len(predictors_tr)

1997772

In [51]:
length_maxtrix = len(vocabulary)+1

embedding_matrix = np.zeros((length_maxtrix, 300))

for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
            continue
    try:        
        embedding_vector = word_vectors[word]
        # words not found in embedding index will be all-zeros.
        if embedding_vector is not None:  
            embedding_matrix[i] = embedding_vector
    except:
        pass

In [52]:
embedding_matrix.shape

(44012, 300)

In [53]:
WV_DIM = 300

In [56]:
model_gen = Sequential()
model_gen.add(Embedding(MAX_NB_WORDS +1,
                     WV_DIM,
                     mask_zero=False,
                     weights=[embedding_matrix],
                     input_length=MAX_SEQUENCE_LENGTH-1,
                     trainable=False))

model_gen.add(GRU(200))
model_gen.add(Dropout(0.2))
model_gen.add(BatchNormalization())
model_gen.add(Dense(MAX_NB_WORDS+1, activation = 'softmax'))

model_gen.compile(loss='categorical_crossentropy', 
                   optimizer = 'adam', 
                   metrics = ['accuracy']) 

In [57]:
model_gen.fit(predictors_tr[:10000], label_tr[:10000], 
               epochs = 100, 
               verbose=1,
               batch_size=50,
               shuffle=True,
               validation_data = (predictors_test[:1000], label_test[:1000]))

Train on 10000 samples, validate on 1000 samples
Epoch 1/100
10000/10000 [==============================] - 46s 5ms/step - loss: 9.1384 - acc: 0.0587 - val_loss: 7.7258 - val_acc: 0.0920
Epoch 2/100
10000/10000 [==============================] - 43s 4ms/step - loss: 6.5110 - acc: 0.1282 - val_loss: 7.4694 - val_acc: 0.1450
Epoch 3/100
10000/10000 [==============================] - 42s 4ms/step - loss: 6.0257 - acc: 0.1674 - val_loss: 6.9321 - val_acc: 0.1620
Epoch 4/100
10000/10000 [==============================] - 44s 4ms/step - loss: 5.6784 - acc: 0.1961 - val_loss: 7.0421 - val_acc: 0.1870
Epoch 5/100
10000/10000 [==============================] - 48s 5ms/step - loss: 5.3889 - acc: 0.2094 - val_loss: 6.9218 - val_acc: 0.2060
Epoch 6/100
10000/10000 [==============================] - 47s 5ms/step - loss: 5.1308 - acc: 0.2208 - val_loss: 7.0327 - val_acc: 0.2170
Epoch 7/100
10000/10000 [==============================] - 47s 5ms/step - loss: 4.8888 - acc: 0.2337 - val_loss: 7.1489 - v

10000/10000 [==============================] - 44s 4ms/step - loss: 0.3963 - acc: 0.8919 - val_loss: 8.4178 - val_acc: 0.2970
Epoch 60/100
10000/10000 [==============================] - 43s 4ms/step - loss: 0.3941 - acc: 0.8910 - val_loss: 8.5690 - val_acc: 0.2710
Epoch 61/100
10000/10000 [==============================] - 44s 4ms/step - loss: 0.3798 - acc: 0.8951 - val_loss: 8.5909 - val_acc: 0.2900
Epoch 62/100
10000/10000 [==============================] - 46s 5ms/step - loss: 0.3933 - acc: 0.8911 - val_loss: 8.6827 - val_acc: 0.2830
Epoch 63/100
10000/10000 [==============================] - 46s 5ms/step - loss: 0.3817 - acc: 0.8931 - val_loss: 8.5605 - val_acc: 0.2890
Epoch 64/100
10000/10000 [==============================] - 46s 5ms/step - loss: 0.3747 - acc: 0.9000 - val_loss: 8.5023 - val_acc: 0.2890
Epoch 65/100
10000/10000 [==============================] - 45s 4ms/step - loss: 0.3791 - acc: 0.8953 - val_loss: 8.6405 - val_acc: 0.2860
Epoch 66/100
10000/10000 [==============

In [61]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    output_sequence = ''
    for _ in range(next_words):
        seed_text = seed_text.lower()
        seed_list = seed_text.split()
        token_list = [word_index.get(word)for word in seed_list]
        token_list= pad_sequences([token_list], maxlen=MAX_SEQUENCE_LENGTH-1, padding="pre")
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    for word in seed_list:
        output_sequence+=word+ ' '
    return output_sequence.strip()

In [62]:
generate_text('с', 15, model_gen, 30)

'с нами покупка домена просто регистрация за 5 минут белее 100 зон от 220 вольт'

In [63]:
generate_text('двери', 10, model_gen, 30)

'двери купе в гардеробную от производителя в краснодаре низкие цены'

In [64]:
model_gen_larger = Sequential()
model_gen_larger.add(Embedding(MAX_NB_WORDS +1,
                     WV_DIM,
                     mask_zero=False,
                     weights=[embedding_matrix],
                     input_length=MAX_SEQUENCE_LENGTH-1,
                     trainable=False))

model_gen_larger.add(GRU(100, return_sequences=True))
model_gen_larger.add(Dropout(0.2))
model_gen_larger.add(GRU(100))
model_gen_larger.add(Dropout(0.2))
model_gen_larger.add(BatchNormalization())
model_gen_larger.add(Dense(MAX_NB_WORDS+1, activation = 'softmax'))

model_gen_larger.compile(loss='categorical_crossentropy', 
                   optimizer = 'adam', 
                   metrics = ['accuracy']) 

In [ ]:
model_gen_larger.fit(predictors_tr[:200000], label_tr[:200000], 
               epochs = 30, 
               verbose=1,
               batch_size=30,
               shuffle=True,
               validation_data = (predictors_test[:1000], label_test[:1000]))

Train on 200000 samples, validate on 1000 samples
Epoch 1/30
    30/200000 [..............................] - ETA: 3:28:17 - loss: 10.6923 - acc: 0.0000e+00